In [2]:
import pandas as pd
%matplotlib inline

In [4]:
stream_path = r"C:\Users\anees\Documents\CS\MP\datasets\comment_stream_117k.csv"
mod_path = r"C:\Users\anees\Documents\CS\MP\datasets\top_week.csv"

In [5]:
stream_df = pd.read_csv(stream_path)
mod_df = pd.read_csv(mod_path)

In [6]:
stream_df.head()

,comment,subreddit
0,"Yeah, my library doesn't have DVD's. Books and...",funny
1,yes yes yes. I don't understand why more peopl...,ufl
2,So dreamy.,askgaybros
3,Thank you! First painting i'm feeling proud of...,apexlegends
4,Grammar,PewdiepieSubmissions


In [7]:
mod_df.head()

,subreddit,comment,score
0,gifs,You can pinpoint the precise moment he is told...,12141.0
1,gifs,Don't know what the brand of pizza it is but i...,14051.0
2,gifs,That look off screen when someone must be tell...,19301.0
3,gifs,Haha! I was watching this kid at the hour rece...,2268.0
4,gifs,Can't wait till he's on Ellen and gets a lifet...,1800.0


In [14]:
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import pickle

In [9]:
classifiers = []
for i in range(6):
    classifiers.append(joblib.load("lr_char_" + str(i) + ".joblib"))

In [12]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

In [15]:
word_vectorizer = pickle.load(open("word_vectorizer.pickle",'rb'))
char_vectorizer = pickle.load(open("char_vectorizer.pickle",'rb'))

In [37]:
print("generating word features")
stream_df_wordfeat = word_vectorizer.transform(stream_df.comment)
print("creating char features")
stream_df_charfeat = char_vectorizer.transform(stream_df.comment)
print("done")
stream_df_feat = hstack([stream_df_charfeat,stream_df_wordfeat])

generating word features
creating char features
done


In [27]:
mod_df = mod_df.dropna()

In [38]:
print("generating word features")
mod_df_wordfeat = word_vectorizer.transform(mod_df.comment)
print("creating char features")
mod_df_charfeat = char_vectorizer.transform(mod_df.comment)
print("done")
mod_df_feat = hstack([mod_df_charfeat,mod_df_wordfeat])

generating word features
creating char features
done


In [30]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [35]:
mod_df['toxic'] = 0
mod_df['severe_toxic'] = 0
mod_df['obscene'] = 0
mod_df['threat'] = 0
mod_df['insult'] = 0
mod_df['identity_hate'] = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [36]:
stream_df['toxic'] = 0
stream_df['severe_toxic'] = 0
stream_df['obscene'] = 0
stream_df['threat'] = 0
stream_df['insult'] = 0
stream_df['identity_hate'] = 0

In [39]:
i = 0
for class_name in class_names:
    mod_df[class_name] = classifiers[i].predict_proba(mod_df_feat)[:,1]
    i+=1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
i = 0
for class_name in class_names:
    stream_df[class_name] = classifiers[i].predict_proba(stream_df_feat)[:,1]
    i+=1

In [41]:
stream_df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,117242.000000,117242.000000,117242.000000,117242.000000,117242.000000,117242.000000
mean,0.122573,0.009505,0.062581,0.003517,0.050330,0.009889
std,0.156798,0.028083,0.130044,0.004244,0.085787,0.012836
min,0.002468,0.000824,0.002985,0.000846,0.002087,0.001552
25%,0.048530,0.004818,0.023183,0.002597,0.021627,0.006613
50%,0.074789,0.006279,0.031806,0.003070,0.030463,0.008282
75%,0.121335,0.008209,0.045620,0.003630,0.044849,0.010313
max,0.999995,0.908390,0.999999,0.513580,0.999074,0.883107


In [42]:
mod_df.describe()

,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,187619.000000,187619.000000,187619.000000,187619.000000,187619.000000,187619.000000,187619.000000
mean,158.301222,0.132891,0.009805,0.067492,0.003609,0.053565,0.010393
std,1217.023686,0.158505,0.025606,0.132858,0.004545,0.083757,0.012456
min,-206.000000,0.004003,0.001209,0.003313,0.001012,0.003262,0.001862
25%,2.000000,0.055139,0.005180,0.025215,0.002668,0.023799,0.007073
50%,4.000000,0.082274,0.006621,0.033962,0.003123,0.032825,0.008714
75%,22.000000,0.133699,0.008568,0.049083,0.003679,0.048786,0.010867
max,74620.000000,0.999993,0.874163,0.999992,0.611907,0.999546,0.777843
